In [8]:
%pip install transformers
%pip install torch


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import torch

In [10]:
clean_data = pd.read_csv("dataset/dataset.csv")
clean_data.head()


,tweet_text,image_path,labels
0,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,dataset/images/0.jpg,"[4, 1, 3]"
1,My horses are retarded https://t.co/HYhqc6d5WN,dataset/images/1.jpg,"[5, 5, 5]"
2,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,dataset/images/2.jpg,"[0, 0, 0]"
3,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,dataset/images/3.jpg,"[1, 0, 0]"
4,“EVERYbody calling you Nigger now!” https://t....,dataset/images/4.jpg,"[1, 0, 1]"


In [11]:
import re
import pandas as pd
import unicodedata

def clean_tweet_text(text):
    # Normalize unicode characters to convert any non-standard variants
    text = unicodedata.normalize('NFKC', text)
    
    # Convert text to lowercase first (to avoid any case sensitivity issues)
    text = text.lower()
    
    # Remove tweet links (URLs starting with https://)
    text = re.sub(r'https://\S+', '', text)
    
    # Remove mentions completely.
    # This pattern matches an "@" mention anywhere in the text, 
    # including at the start, middle, or end of the string
    text = re.sub(r'@\w+', '', text)
    
    # Remove punctuation (retain only alphanumerics and spaces)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra whitespace (including newlines and tabs)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply the cleaning function to your DataFrame column
clean_data["tweet_text"] = clean_data["tweet_text"].apply(clean_tweet_text)

# Save the updated dataset if needed
clean_data.to_csv("dataset/clean_dataset.csv", index=False)

In [12]:
clean_data = pd.read_csv("dataset/clean_dataset.csv")
clean_data.head()


,tweet_text,image_path,labels
0,nigga,dataset/images/0.jpg,"[4, 1, 3]"
1,my horses are retarded,dataset/images/1.jpg,"[5, 5, 5]"
2,nigga on ma momma youngboy be spitting real sh...,dataset/images/2.jpg,"[0, 0, 0]"
3,rt xxsugvngxx i ran into this holy nigga today,dataset/images/3.jpg,"[1, 0, 0]"
4,everybody calling you nigger now,dataset/images/4.jpg,"[1, 0, 1]"


# Text data tokenization

In [13]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Function to tokenize text
def tokenize_text(text):
    inputs = tokenizer(
        text,
        padding="max_length",  # Pad to a fixed length
        truncation=True,       # Truncate if the text is too long
        max_length=128,        # Maximum length for BERT
        return_tensors="pt"    # Return PyTorch tensors
    )
    return inputs["input_ids"].squeeze(0), inputs["attention_mask"].squeeze(0)

# Apply tokenization to the dataset
clean_data["input_ids"], clean_data["attention_mask"] = zip(*clean_data["tweet_text"].apply(tokenize_text))

# Save the tokenized dataset (optional)
clean_data.to_csv("dataset/dataset_tokenized.csv", index=False)

In [14]:
clean_data = pd.read_csv("dataset/dataset_tokenized.csv")
clean_data.head()


,tweet_text,image_path,labels,input_ids,attention_mask
0,nigga,dataset/images/0.jpg,"[4, 1, 3]","tensor([ 101, 9152, 23033, 102, 0, ...","tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,my horses are retarded,dataset/images/1.jpg,"[5, 5, 5]","tensor([ 101, 2026, 5194, 2024, 2128, 7559, 57...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,..."
2,nigga on ma momma youngboy be spitting real sh...,dataset/images/2.jpg,"[0, 0, 0]","tensor([ 101, 9152, 23033, 2006, 5003, 236...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
3,rt xxsugvngxx i ran into this holy nigga today,dataset/images/3.jpg,"[1, 0, 0]","tensor([ 101, 19387, 22038, 6342, 2290, 160...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
4,everybody calling you nigger now,dataset/images/4.jpg,"[1, 0, 1]","tensor([ 101, 7955, 4214, 2017, 9152, 133...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,..."
